### ЭТОТ НОУТБУК ВЫПОЛНЯЛСЯ ВТОРЫМ ###

После выполнения первого обучения в ноутбуке Conditional_sampling/

In [ ]:
import torch
import wandb

from default_mnist_config import create_default_mnist_config
from diffusion import DiffusionRunner
from models.classifier import ResNet, ResidualBlock, ConditionalResNet
from data_generator import DataGenerator

from tqdm.auto import trange

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
pip install wandb

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 64.7 MB/s 
     |████████████████████████████████| 144 kB 64.3 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=04399ed8b9f57f3a2feb1b897fc18132ba6a9c860fab04d08ecb1e57ebde6bb5
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os

In [7]:
os.listdir('./')

['.config', 'drive', 'sample_data']

In [8]:
import os
os.chdir("./drive/MyDrive/Colab Notebooks")

In [9]:
os.chdir("./ddpm_hw_clean")

In [3]:
pip install ml_collections

     |████████████████████████████████| 77 kB 3.5 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=9e8702c5d4a6154fd1af9266d166609647bfa7ea3dcedde5cb25faae88c6fff7
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections


In [11]:
#import torch
#import wandb

#from default_mnist_config import create_default_mnist_config
#from diffusion import DiffusionRunner
#from models.classifier import ResNet, ResidualBlock, ConditionalResNet
#from data_generator import DataGenerator

#from tqdm.auto import trange

#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [12]:
device = torch.device('cuda')
classifier_args = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2]
}
model = ResNet(**classifier_args)
model.to(device)

optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_func = torch.nn.CrossEntropyLoss()

In [13]:
datagen = DataGenerator(create_default_mnist_config())
train_generator = datagen.sample_train()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [14]:
TOTAL_ITERS = 2_000
EVAL_FREQ = 500

### Обучите классификатор только на чистых картинках. Он понадобится нам для классификации условно сгенерированных картинок

In [15]:
model.train()

for iter_idx in trange(1, 1 + TOTAL_ITERS):
    
    """
    train
    """
    ####
    (X, y) = next(train_generator)


    X = X.to(device)
    y = y.to(device)

    labels_p = model(X)
    loss = loss_func(labels_p, y)

    optim.zero_grad()
    loss.backward()
    optim.step()
    
    ####
    if iter_idx % EVAL_FREQ == 0:
        valid_accuracy = 0
        """
        validate
        """
        ####
        cnt = 0

        model.eval()
        with torch.no_grad():
            """
            validate classifier
            """
            for (X, y) in datagen.valid_loader:

                X = X.to(device)
                y = y.to(device)

                labels_p = model(X)

                valid_accuracy += torch.sum((torch.argmax(labels_p, dim=1) == y))
                cnt += X.size(0)

        valid_accuracy = valid_accuracy / cnt
        print('Clean MNIST classifier\'s accuracy:', valid_accuracy)
        model.train()
        
      ###
    
        #print('Clean MNIST classifier\'s accuracy:', valid_accuracy)
model.eval()

  0%|          | 0/2000 [00:00<?, ?it/s]

Clean MNIST classifier's accuracy: tensor(0.9785, device='cuda:0')
Clean MNIST classifier's accuracy: tensor(0.9844, device='cuda:0')
Clean MNIST classifier's accuracy: tensor(0.9924, device='cuda:0')
Clean MNIST classifier's accuracy: tensor(0.9902, device='cuda:0')


ResNet(
  (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

In [16]:
torch.save(model.state_dict(), './ddpm_checkpoints/clean_classifier.pth')

###После этого выполнялся ноутбук CALC_FID###